In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import tensorflow as tf
from pathlib import Path
from skimage import io

from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# import keras library
import keras

# import Sequential from the keras models module
from keras import Sequential

# import Dense, Dropout, Flatten, Conv2D, MaxPooling2D from the keras layers module
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
def image_loader(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)
    
    return image

In [3]:
def dataset_preprocessing(dataset_path, labels_file_path, image_size, image_paths_pickle):
    with open(labels_file_path, "r") as f:
        classes = f.read().split("\n")
       
    images = []
    labels = []
    image_paths = []
    print(classes)
    
    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)
            images.append(image_loader(image_path, image_size))
            image_paths.append(image_path)
            
            for idx in range(len(classes)):
                if classes[idx] in image_name:
                    labels.append(idx)
                    break
                elif len(classes) == idx:
                    labels.append(len(classes))
                    break
        except:
            pass
        
    with open(image_paths_pickle + ".pickle", "wb") as f:
        pickle.dump(image_paths, f)
    
   # assert len(images) == len(labels)
    return np.array(images), np.array(labels)

In [4]:
dataset_path = "JPEGImages/"
label_path = "labels.txt"
image_size = (120,72)

In [5]:
X, y = dataset_preprocessing(dataset_path, label_path, image_size, "test_image_pickle")

['bus', 'bicycle', 'car', 'motorbike', 'pedestrian', 'trafficlight', 'trafficsignal', '']


In [6]:
X.shape

(106920, 72, 120, 3)

In [7]:
y.shape

(106920,)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

In [9]:
# set model constants
num_classes = 1

# define model as Sequential
model = Sequential()

# first convolutional layer with 32 filters
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(72, 120, 3)))

# add a second 2D convolutional layer with 64 filters
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

In [10]:
# reduce dimensionality through max pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# third convolutional layer with 64 filters
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# add dropout to prevent over fitting
model.add(Dropout(0.25))
# necessary flatten step preceeding dense layer
model.add(Flatten())
# fully connected layer
model.add(Dense(128, activation='relu'))

# add additional dropout to prevent overfitting
model.add(Dropout(0.5))

# prediction layers
model.add(Dense(num_classes, activation='sigmoid', name='preds'))

# show model summary
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 70, 118, 32)       896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 68, 116, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 58, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 56, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 56, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 114688)            0         
________________________

In [ ]:
model.compile(
    # set the loss as binary_crossentropy
    loss='binary_crossentropy',
    # set the optimizer as stochastic gradient descent
    optimizer=keras.optimizers.SGD(lr=0.001),
    # set the metric as accuracy
    metrics=['accuracy']
)

# mock-train the model using the first ten observations of the train and test sets
model.fit(
    X_train,
    y_train,
    epochs=5,
    verbose=1,
    validation_data=(X_test, y_test)
)

In [12]:
# evaluate model on test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 44.70917298773278
Test accuracy: 0.539356528247647
